# Tutorial for AFMpy.LAFM

## Imports

In [ ]:
# Standard library imports
import json
import logging

# Third party imports
import matplotlib.pyplot as plt

# AFMpy imports
from AFMpy import Stack, Plotting

## Logging Configuration

Each module in AFMpy contains logging to for debugging purposes via the default python logging library. Logging for the modules should always be configured at the application level. Included in these tutorials are example logging configuration files that can be loaded with the following functions. You may adjust these logging configuration files as you see fit.

In [ ]:
# Load the preconfigured logging settings
with open('logs/LAFM_Tutorial_LoggingConfig.json', 'r') as f:
    LOGGING_CONFIG = json.load(f)

# Set up the logging configuration
logging.config.dictConfig(LOGGING_CONFIG)

## Matplotlib Configuration

Included within the ```Plotting``` module are functions for creating the high quality figures. A default configuration that matches the figures in the publication is activated by running the following function.

In [ ]:
# Configure the rcParams for the plots
Plotting.configure_formatting()

## Load the Stacks

Here we demonstrate how to load a compressed pickle file containing stack information. Recall from the ```SimAFM_Tutorial``` that stack pickles should be distributed alongside a cryptographic signature (```.sig```) and public key (```.pub```). To load the file call ```Stack.load_compressed_pickle``` specifying the ```pickle_filepath``` to the stack and the ```public_key_filepath``` (both ```str``` or ```Path```). This method will

1. Compute the cryptographic hash of the ```.xz``` file
2. Verify the hash against the corresponding ```.sig``` file using the public key.
3. Proceed to load an instance of the ```Stack``` object **only if** the signature is valid.

If the signature verification fails an exception will be raised. Verification can be bypassed by setting the ```verify_pickle``` (```bool```) to ```False```. However, this practice is **not** recommended. Disabling signature checks leaves you vulnerable to malicious files—use it only at your own risk.

In [ ]:
# Set the filepath for the public key to verify the integrity of the stacks.
PUBLIC_KEY_FILEPATH = '../common/keys/Tutorial_Public.pub'

# Load the cytoplasmic and periplasmic stacks
cytoplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AC-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

periplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AP-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

The ```Stack``` objects contain metadata related to the scanning simulation. They can be displayed with the ```Stack.display_metadata``` method.

In [ ]:
print('Cytoplasmic Stack Metadata:')
cytoplasmic_stack.display_metadata()
print('Periplasmic Stack Metadata:')
periplasmic_stack.display_metadata()

## Processing the Stacks

The ```Stack.calc_mean_image``` method computes the per‑pixel average of the entire stack with NumPy, stores the result in ```stack.mean_image```, and returns it. If ```stack.mean_image``` already exists the cached version is returned, unless you pass ```force_recalc=True```.

In [ ]:
# Calculating the Mean image for each stack.
cytoplasmic_mean_image = cytoplasmic_stack.calc_mean_image()

periplasmic_mean_image = periplasmic_stack.calc_mean_image()

The ```Stack.calc_LAFM_image``` method computes the LAFM image, stores the result in ```stack.LAFM_image```, and returns it. If ```stack.LAFM_image``` already exists the cached version is returned, unless you pass ```force_recalc=True```. ```target_resolution``` (```tuple[int,int]```) defines the output resolution that the LAFM image should be upscaled to; ```target_resolution``` must be greater than the resolution of the input stack. ```sigma``` (```float```) is the width of the gaussian broadening applied to each detected local maxima.

In [ ]:
# Set the target resolution and gaussian width for LAFM processing.
target_resolution = (96,96)
sigma = 2.25

# Computer the LAFM images.
cytoplasmic_lafm_image = cytoplasmic_stack.calc_LAFM_image(target_resolution = target_resolution, sigma = sigma)
periplasmic_lafm_image = periplasmic_stack.calc_LAFM_image(target_resolution = target_resolution, sigma = sigma)

## Plot the Images

Finally, plotting these images is similar to plotting a single frame. For each function ```Stack.plot_mean_image``` or ```Stack.plot_LAFM_image``` pass the axis to draw the image to and any additional keyword arguments like ```cmap```.

Here we use an alternate method of drawing the colorbar. Automatically adding the colorbar to the right column axes improperly resizes the plots. Here we create new axes objects to the right of the images and manually set the size. Once they look appropriately sized, we can draw the colorbar directly to the axes with ```Plotting.draw_colorbar_to_ax```, which takes the minimum and maximum height values, the colormap, and the axis to draw to as arguments.

In [ ]:
fig, ax = plt.subplots(2,2, figsize = (8,8))
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])

# Plot the mean and LAFM images for both stacks with the scalebar.
# Scalebars are 3x length for LAFM, because of the resoultion increase.
cytoplasmic_stack.plot_mean_image(ax[0,0], cmap=Plotting.LAFMcmap)
Plotting.add_scalebar(10/cytoplasmic_stack.resolution, label = '1nm', ax = ax[0,0])
periplasmic_stack.plot_mean_image(ax[1,0], cmap=Plotting.LAFMcmap)
Plotting.add_scalebar(10/periplasmic_stack.resolution, label = '1nm', ax = ax[1,0])
cytoplasmic_stack.plot_LAFM_image(ax[0,1], cmap=Plotting.LAFMcmap)
Plotting.add_scalebar(30/cytoplasmic_stack.resolution, label = '1nm', size_vertical = 3/8, ax = ax[0,1])
periplasmic_stack.plot_LAFM_image(ax[1,1], cmap=Plotting.LAFMcmap)
Plotting.add_scalebar(30/periplasmic_stack.resolution, label = '1nm', size_vertical = 3/8, ax = ax[1,1])

# Add the colorbars to the right of the images.
cbar_ax = fig.add_axes([0.91, 0.5, 0.02, 0.38])
Plotting.draw_colorbar_to_ax(0, cytoplasmic_stack.LAFM_image.max(), Plotting.LAFMcmap,
                             label = 'Height (Å)', cbar_ax = cbar_ax)
cbar_ax = fig.add_axes([0.91, 0.11, 0.02, 0.38])
Plotting.draw_colorbar_to_ax(0, periplasmic_stack.LAFM_image.max(), Plotting.LAFMcmap,
                             label = 'Height (Å)', cbar_ax = cbar_ax)

ax[0,0].set_title('Mean Image', fontsize = 16)
ax[0,0].set_ylabel('Cytoplasmic', fontsize = 16)
ax[0,1].set_title('LAFM Image', fontsize = 16)
ax[1,0].set_ylabel('Periplasmic', fontsize = 16)

plt.show()